In [33]:
import numpy as np
import heapq
from collections import namedtuple
from random import choice
from icecream import ic
import itertools
import time

In [34]:
PUZZLE_DIM = 7
Action = namedtuple('Action', ['pos1', 'pos2'])

In [35]:
def available_actions(state: np.ndarray) -> list[Action]:
    # Find the position of zero (empty space)
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = []
    
    # Optimisation: use a list of directions instead of separate ifs
    directions = [
        (-1, 0),  # su
        (1, 0),   # giù
        (0, -1),  # sinistra
        (0, 1)    # destra
    ]
    
    for dx, dy in directions:
        new_x, new_y = x + dx, y + dy
        if 0 <= new_x < PUZZLE_DIM and 0 <= new_y < PUZZLE_DIM:
            actions.append(Action((x, y), (new_x, new_y)))
            
    return actions

def do_action(state: np.ndarray, act: Action) -> np.ndarray:
    new_state = state.copy()
    new_state[act.pos1], new_state[act.pos2] = new_state[act.pos2], new_state[act.pos1]
    return new_state

In [36]:
def heuristic(state: np.ndarray)-> int:
    # Manhattan distance for each puzzle piece
    distance=0
    goal_state=np.array([i for i in range(1,PUZZLE_DIM**2)]+[0]).reshape((PUZZLE_DIM,PUZZLE_DIM)) #values in ascending order, with 0 in the last position
    target_positions = {
        value: divmod(value-1, PUZZLE_DIM) 
        for value in range(1, PUZZLE_DIM**2)
    }
    
    for x in range(PUZZLE_DIM):
        for y in range(PUZZLE_DIM):
            value = state[x, y]
            if value != 0:  # ignore empty space
                target_x, target_y = target_positions[value]
                distance += abs(x - target_x) + abs(y - target_y)
                
    return distance

In [37]:
def a_star(initial_state: np.ndarray) -> list[Action]:
    open_set = []
    counter = itertools.count()  # Counter for differentiating states in the heap
    heapq.heappush(open_set, (0, next(counter), initial_state, []))  # (priorità, conteggio, stato, percorso)
    visited = set()
    
    while open_set:
        cost, _, state, path = heapq.heappop(open_set)
        
        # Check if we have reached the target status
        if np.array_equal(state, np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))):
            return path,state
        
        visited.add(state.tobytes())
        
        # Generate neighbouring states
        for act in available_actions(state):
            new_state = do_action(state, act)
            if new_state.tobytes() not in visited:
                new_cost = len(path) + 1 + heuristic(new_state)
                heapq.heappush(open_set, (new_cost, next(counter), new_state, path + [act]))
    
    return None

In [38]:
def generate_puzzle(num_moves=50):
    state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
    
    for _ in range(num_moves):
        acts = available_actions(state)
        state = do_action(state, choice(acts))
    
    return state

In [39]:
state = generate_puzzle(100)
ic(state)
None

ic| state: array([[ 1,  2, 10,  3,  5,  7,  0],
                  [ 8,  9,  4, 11, 12,  6, 21],
                  [15, 16, 17, 18, 19, 14, 13],
                  [22, 23, 25, 32, 27, 40, 28],
                  [29, 30, 24, 34, 33, 26, 20],
                  [36, 37, 31, 39, 41, 42, 35],
                  [43, 44, 38, 45, 46, 47, 48]])


In [40]:
start_time = time.time()
solution_path = a_star(state)
end_time  = time.time()

if solution_path:
    print(f"\nSolution found in {end_time - start_time:.2f} seconds!")
    print("Solution found:", solution_path[0])
    print("Number of moves:", len(solution_path[0]))
    ic(solution_path[1])
else:
    print("Solution not found.")

ic| solution_path[1]: array([[ 1,  2,  3,  4,  5,  6,  7],
                             [ 8,  9, 10, 11, 12, 13, 14],
                             [15, 16, 17, 18, 19, 20, 21],
                             [22, 23, 24, 25, 26, 27, 28],
                             [29, 30, 31, 32, 33, 34, 35],
                             [36, 37, 38, 39, 40, 41, 42],
                             [43, 44, 45, 46, 47, 48,  0]])



Solution found in 2.81 seconds!
Solution found: [Action(pos1=(0, 6), pos2=(0, 5)), Action(pos1=(0, 5), pos2=(1, 5)), Action(pos1=(1, 5), pos2=(2, 5)), Action(pos1=(2, 5), pos2=(2, 6)), Action(pos1=(2, 6), pos2=(3, 6)), Action(pos1=(3, 6), pos2=(4, 6)), Action(pos1=(4, 6), pos2=(5, 6)), Action(pos1=(5, 6), pos2=(5, 5)), Action(pos1=(5, 5), pos2=(5, 4)), Action(pos1=(5, 4), pos2=(4, 4)), Action(pos1=(4, 4), pos2=(4, 5)), Action(pos1=(4, 5), pos2=(3, 5)), Action(pos1=(3, 5), pos2=(3, 6)), Action(pos1=(3, 6), pos2=(2, 6)), Action(pos1=(2, 6), pos2=(1, 6)), Action(pos1=(1, 6), pos2=(1, 5)), Action(pos1=(1, 5), pos2=(2, 5)), Action(pos1=(2, 5), pos2=(3, 5)), Action(pos1=(3, 5), pos2=(3, 4)), Action(pos1=(3, 4), pos2=(4, 4)), Action(pos1=(4, 4), pos2=(4, 3)), Action(pos1=(4, 3), pos2=(3, 3)), Action(pos1=(3, 3), pos2=(2, 3)), Action(pos1=(2, 3), pos2=(1, 3)), Action(pos1=(1, 3), pos2=(1, 2)), Action(pos1=(1, 2), pos2=(0, 2)), Action(pos1=(0, 2), pos2=(0, 3)), Action(pos1=(0, 3), pos2=(1, 3))